In [2]:
import pandas as pd
from urllib.parse import urlparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
!pip install validators==0.20.0
import validators
import os.path
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split




# Load the Dataset
#df = pd.read_csv('../CSCT Masters Project/Datasets I Used/List of URLs.csv')
data = pd.read_csv('List of URLs.csv')
#data.head()
print(data.shape)
data.type.value_counts()

(196564, 2)


,count
type,
benign,143716
defacement,36368
phishing,11794
malware,4685


In [3]:
#count of missing values in each column

missing = data.isnull().sum()
print(f"Number of missing values: {missing}")


before_duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {before_duplicates}")
df = data.drop_duplicates()
after_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows after removing duplicates: {after_duplicates}")


Number of missing values: url     0
type    1
dtype: int64
Number of duplicate rows: 2872
Number of duplicate rows after removing duplicates: 0


In [4]:
print(df.shape)
df.type.value_counts()

(193692, 2)


,count
type,
benign,143714
defacement,36193
phishing,11793
malware,1991


In [5]:
df['length of url'] = df['url'].apply(lambda i: len(str(i)))

def having_ip_address(url):
  return 1 if validators.ip_address.ipv4(url) or validators.ip_address.ipv6(url) else 0

df['use of ip'] = df['url'].apply(lambda i: having_ip_address(i))
df.head()

<ipython-input-5-ad8f9da966d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length of url'] = df['url'].apply(lambda i: len(str(i)))
<ipython-input-5-ad8f9da966d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['use of ip'] = df['url'].apply(lambda i: having_ip_address(i))


,url,type,length of url,use of ip
0,br-icloud.com.br,phishing,16,0
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0


In [6]:
def use_of_hostname(url):
    hostname = urlparse(url).hostname
    if hostname is None:  # Handle the case when hostname is None
        return 0
    return 1 if hostname in url else 0

df['use of hostname'] = df['url'].apply(use_of_hostname)
df['length of hostname'] = df['url'].apply(lambda i: len(urlparse(i).netloc))

df.head()

<ipython-input-6-cb628977f393>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['use of hostname'] = df['url'].apply(use_of_hostname)
<ipython-input-6-cb628977f393>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length of hostname'] = df['url'].apply(lambda i: len(urlparse(i).netloc))


,url,type,length of url,use of ip,use of hostname,length of hostname
0,br-icloud.com.br,phishing,16,0,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0,0,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0,1,21
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0,1,23


In [8]:
def count_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df['count of /'] = df['url'].apply(lambda i: count_of_dir(i))

def count_of_SchemeSeparator(url):
    urldir = urlparse(url).path
    return urldir.count('//')
df['count of //'] = df['url'].apply(lambda i: count_of_SchemeSeparator(i))

<ipython-input-8-063ea8444847>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of /'] = df['url'].apply(lambda i: count_of_dir(i))
<ipython-input-8-063ea8444847>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of //'] = df['url'].apply(lambda i: count_of_SchemeSeparator(i))


In [9]:
# Length of First Directory
def fd_length(url):
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    if len(path_parts) > 1:
        return len(path_parts[1])
    else:
        return 0
df['length of first directory'] = df['url'].apply(fd_length)

# Length of Top Level Domain
def tld_length(url):
    parsed_url = urlparse(url)
    return len(parsed_url.netloc)
df['length of top level domain'] = df['url'].apply(tld_length)

df.head()

<ipython-input-9-7f2584ad75bd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length of first directory'] = df['url'].apply(fd_length)
<ipython-input-9-7f2584ad75bd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length of top level domain'] = df['url'].apply(tld_length)


,url,type,length of url,use of ip,use of hostname,length of hostname,count of /,count of //,length of first directory,length of top level domain
0,br-icloud.com.br,phishing,16,0,0,0,0,0,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0,0,0,2,0,5,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,0,3,0,7,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0,1,21,1,0,9,21
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0,1,23,1,0,9,23


In [10]:
def count_of_subdomains(url):
  parsed_url = urlparse(url)
  # Handling the case where hostname is None
  if parsed_url.hostname is None:
    return 0  # return 0 when a url is missing hostname
  subdomains = parsed_url.hostname.split('.')
  return len(subdomains) - 2

df['count of subdomains'] = df['url'].apply(count_of_subdomains)

df.head()

<ipython-input-10-64a86a89dd15>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of subdomains'] = df['url'].apply(count_of_subdomains)


,url,type,length of url,use of ip,use of hostname,length of hostname,count of /,count of //,length of first directory,length of top level domain,count of subdomains
0,br-icloud.com.br,phishing,16,0,0,0,0,0,0,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0,0,0,2,0,5,0,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,0,3,0,7,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0,1,21,1,0,9,21,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0,1,23,1,0,9,23,0


In [11]:
# checking whether a url shortening service has been used in the url or not

shorteners = [
    "bit.ly", "goo.gl", "shorte.st", "go2l.ink", "x.co", "ow.ly", "t.co",
    "tinyurl", "tr.im", "is.gd", "cli.gs", "yfrog.com", "migre.me",
    "ff.im", "tiny.cc", "url4.eu", "twit.ac", "su.pr", "twurl.nl",
    "snipurl.com", "short.to", "BudURL.com", "ping.fm", "post.ly",
    "Just.as", "bkite.com", "snipr.com", "fic.kr", "loopt.us",
    "doiop.com", "short.ie", "kl.am", "wp.me", "rubyurl.com", "om.ly",
    "to.ly", "bit.do", "t.co", "lnkd.in", "db.tt", "qr.ae", "adf.ly",
    "goo.gl", "bitly.com", "cur.lv", "tinyurl.com", "ow.ly", "bit.ly",
    "ity.im", "q.gs", "is.gd", "po.st", "bc.vc", "twitthis.com", "u.to",
    "j.mp", "buzurl.com", "cutt.us", "u.bb", "yourls.org", "x.co",
    "prettylinkpro.com", "scrnch.me", "filoops.info", "vzturl.com",
    "qr.net", "1url.com", "tweez.me", "v.gd", "tr.im", "link.zip.net"
]

def shortening_service(url):
    for shortener in shorteners:
        if shortener in url:
            return 1
    return 0

df['use of url shortner'] = df['url'].apply(lambda i: shortening_service(i))
df.head()

<ipython-input-11-4105bb3aecbb>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['use of url shortner'] = df['url'].apply(lambda i: shortening_service(i))


,url,type,length of url,use of ip,use of hostname,length of hostname,count of /,count of //,length of first directory,length of top level domain,count of subdomains,use of url shortner
0,br-icloud.com.br,phishing,16,0,0,0,0,0,0,0,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0,0,0,2,0,5,0,0,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,0,3,0,7,0,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0,1,21,1,0,9,21,1,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0,1,23,1,0,9,23,0,0


In [12]:
df['count of https'] = df['url'].apply(lambda i : i.count('https'))
df['count of http'] = df['url'].apply(lambda i : i.count('http'))

<ipython-input-12-d79d28f4ba1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of https'] = df['url'].apply(lambda i : i.count('https'))
<ipython-input-12-d79d28f4ba1e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of http'] = df['url'].apply(lambda i : i.count('http'))


In [13]:
# counting multiple special characters
df['count of %'] = df['url'].apply(lambda i: i.count('%'))
df['count of ?'] = df['url'].apply(lambda i: i.count('?'))
df['count of -'] = df['url'].apply(lambda i: i.count('-'))
df['count of ='] = df['url'].apply(lambda i: i.count('='))
df['count of .'] = df['url'].apply(lambda i: i.count('.'))
df['count of www'] = df['url'].apply(lambda i: i.count('www'))
df['count of @'] = df['url'].apply(lambda i: i.count('@'))
df['count of #'] = df['url'].apply(lambda i: i.count('#'))
df['count of !'] = df['url'].apply(lambda i: i.count('!'))
df['count of $'] = df['url'].apply(lambda i: i.count('$'))
df['count of *'] = df['url'].apply(lambda i: i.count('*'))
df['count of _'] = df['url'].apply(lambda i: i.count('_'))

<ipython-input-13-276df6e90025>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of %'] = df['url'].apply(lambda i: i.count('%'))
<ipython-input-13-276df6e90025>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of ?'] = df['url'].apply(lambda i: i.count('?'))
<ipython-input-13-276df6e90025>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [14]:
df['count of numbers'] = df['url'].str.count('\d')

df['count of letters'] = df['url'].str.count('[a-zA-Z]')
df.head()

<ipython-input-14-fc888fe11e88>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of numbers'] = df['url'].str.count('\d')
<ipython-input-14-fc888fe11e88>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count of letters'] = df['url'].str.count('[a-zA-Z]')


,url,type,length of url,use of ip,use of hostname,length of hostname,count of /,count of //,length of first directory,length of top level domain,...,count of .,count of www,count of @,count of #,count of !,count of $,count of *,count of _,count of numbers,count of letters
0,br-icloud.com.br,phishing,16,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,13
1,mp3raid.com/music/krizz_kaliko.html,benign,35,0,0,0,2,0,5,0,...,2,0,0,0,0,0,0,1,1,29
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,0,3,0,7,0,...,2,0,0,0,0,0,0,0,1,25
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,0,1,21,1,0,9,21,...,3,1,0,0,0,0,0,2,7,63
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,0,1,23,1,0,9,23,...,2,0,0,0,0,0,0,1,22,199


In [15]:
lb_make = LabelEncoder()
df["type_code"] = lb_make.fit_transform(df["type"])
print(df["type_code"].value_counts())
df.to_csv('my_data.csv', index=False)

<ipython-input-15-bcde139437f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["type_code"] = lb_make.fit_transform(df["type"])


type_code
0    143714
1     36193
3     11793
2      1991
4         1
Name: count, dtype: int64


In [16]:
X = df.drop(['type_code', 'url', 'type'], axis=1)  # Predictor variables - all columns except type_code, url, and type
y = df['type_code']              # Target variable


#print("\nTarget variable (y):")
#print(y.head())

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Assuming X and y are already defined

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation scores:", scores)
print("Average cross-validation score:", np.mean(scores))

# Train the model on the entire dataset
rf_classifier.fit(X, y)

# Make predictions on the test set (assuming you have a separate test set)
# Replace X_test with your actual test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy on test set:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.99166215 0.99122332 0.99176519 0.99173938 0.99194589]
Average cross-validation score: 0.9916671852678803

Accuracy on test set: 0.9982704767805054

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28686
           1       1.00      1.00      1.00      7205
           2       1.00      0.84      0.91       391
           3       0.98      1.00      0.99      2457

    accuracy                           1.00     38739
   macro avg       0.99      0.96      0.97     38739
weighted avg       1.00      1.00      1.00     38739


Confusion Matrix:
[[28686     0     0     0]
 [    0  7205     0     0]
 [    3     0   329    59]
 [    0     4     1  2452]]


In [19]:
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

scores = cross_val_score(clf, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

report = classification_report(y_test, y_pred)
print("Classification report:\n", report)

Accuracy: 0.9851054492888304


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.98732543 0.986138   0.98492436 0.98564717 0.98347875]
Average cross-validation score: 0.9855027442643669
Confusion matrix:
 [[28656    18     7     5]
 [   99  7024    10    72]
 [   37    53   197   104]
 [   12   146    14  2285]]
Classification report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     28686
           1       0.97      0.97      0.97      7205
           2       0.86      0.50      0.64       391
           3       0.93      0.93      0.93      2457

    accuracy                           0.99     38739
   macro avg       0.94      0.85      0.88     38739
weighted avg       0.98      0.99      0.98     38739



In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

nb_classifier = GaussianNB()

# Training the model
nb_classifier.fit(X_train, y_train)

# Making predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

scores = cross_val_score(nb_classifier, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

report = classification_report(y_test, y_pred)
print("Classification report:\n", report)

Accuracy: 0.8602441983530809


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.86140582 0.8602442  0.86021478 0.859905   0.86021478]
Average cross-validation score: 0.8603969143992112
Confusion matrix:
 [[25885  1845   811   145]
 [    0  7184    19     2]
 [   82   205    93    11]
 [ 1605   585   104   163]]
Classification report:
               precision    recall  f1-score   support

           0       0.94      0.90      0.92     28686
           1       0.73      1.00      0.84      7205
           2       0.09      0.24      0.13       391
           3       0.51      0.07      0.12      2457

    accuracy                           0.86     38739
   macro avg       0.57      0.55      0.50     38739
weighted avg       0.86      0.86      0.85     38739



In [22]:
import random
import string

def random_url():
    domain = ''.join(random.choices(string.ascii_lowercase, k=8)) + ".com"
    path = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))
    return f"http://{domain}/{path}"

print(random_url())

http://rujstapm.com/6lkftxjnhb


In [25]:
new_url = random_url()
#new_url = input("Enter a URL: ")

new_url_features = {
    'length of url': len(new_url),
    'use of ip': having_ip_address(new_url),
    'use of hostname': use_of_hostname(new_url),
    'length of hostname': len(urlparse(new_url).netloc),
    'count of /': count_of_dir(new_url),
    'count of //': count_of_SchemeSeparator(new_url),
    'length of first directory': fd_length(new_url),
    'length of top level domain': tld_length(new_url),
    'count of subdomains': count_of_subdomains(new_url),
    'use of url shortner': shortening_service(new_url),
    'count of https': new_url.count('https'),
    'count of http': new_url.count('http'),
    'count of %': new_url.count('%'),
    'count of ?': new_url.count('?'),
    'count of -': new_url.count('-'),
    'count of =': new_url.count('='),
    'count of .': new_url.count('.'),
    'count of www': new_url.count('www'),
    'count of @': new_url.count('@'),
    'count of #': new_url.count('#'),
    'count of !': new_url.count('!'),
    'count of $': new_url.count('$'),
    'count of *': new_url.count('*'),
    'count of _': new_url.count('_'),
    'count of numbers': new_url.count('\d'),
    'count of letters': new_url.count('[a-zA-Z]')
}

# Create a DataFrame for the new URL
new_url_df = pd.DataFrame([new_url_features])

# RandomForest Prediction
model = RandomForestClassifier()
model.fit(X_train, y_train)
prediction = model.predict(new_url_df)

# Interpreting the prediction (0 for benign, 1 for defacement, 2 for malware, 3 for phishing)
print("Random Forest Prediction:", prediction)

# Naive Bayes prediction
nb_prediction = nb_classifier.predict(new_url_df)
print("Naive Bayes Prediction:", nb_prediction)

# MLPClassifier prediction
mlp_prediction = clf.predict(new_url_df)
print("MLPClassifier Prediction:", mlp_prediction)

Random Forest Prediction: [1]
Naive Bayes Prediction: [1]
MLPClassifier Prediction: [1]
